In [212]:
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd

In [215]:
from util.preprocess.csi import linear_interpolate_resampling

In [26]:
import importlib
import util
importlib.reload(util)
importlib.reload(util.preprocess.handle_session_data)

<module 'util.preprocess.handle_session_data' from '/workspace/deformation-prediction-multi-environment/util/preprocess/handle_session_data.py'>

In [27]:
from util.preprocess.handle_session_data import convert_jst_to_utc, load_meta_data, get_session_data

In [4]:
root_data_dirpath = Path('/workspace/deformation-prediction-multi-environment/data')
raw_data_dirpath = root_data_dirpath / 'raw'

In [5]:
session_data_filepath = raw_data_dirpath / 'session.csv'

In [6]:
session_df = pd.read_csv(session_data_filepath)

In [7]:
session_df

,session_id,location,start_time_JST,end_time_JST
0,session_20240926_151603,512,2024-09-26 15:17:00,2024-09-26 15:22:00
1,session_20240926_152330,512,2024-09-26 15:24:00,2024-09-26 15:29:00
2,session_20240926_153446,512,2024-09-26 15:35:00,2024-09-26 15:40:00
3,session_20240926_154624,512,2024-09-26 15:47:00,2024-09-26 15:52:00
4,session_20240926_165118,511,2024-09-26 16:52:00,2024-09-26 16:57:00
5,session_20240926_170021,511,2024-09-26 17:01:00,2024-09-26 17:06:00
6,session_20240926_171232,511,2024-09-26 17:13:00,2024-09-26 17:18:00
7,session_20240926_172237,511,2024-09-26 17:23:00,2024-09-26 17:28:00
8,session_20240926_175737,514B,2024-09-26 17:58:00,2024-09-26 18:03:00
9,session_20240926_180441,514B,2024-09-26 18:05:00,2024-09-26 18:10:00


# Filtering

In [244]:
def filter_by_mac(csi_data_df):

    target_mac_addr = '08:D1:F9:34:A5:98'
    target_sig_mode = 1 # 802.11n
    
    # MACアドレスと11nでフィルタリング
    filtered_df = csi_data_df[(csi_data_df['mac_addr'] == target_mac_addr) & (csi_data_df['sig_mode'] == target_sig_mode)][['csi_data', 'timestamp']]
    
    # 'csi_data' の処理
    filtered_df['csi_data'] = filtered_df['csi_data'].str.strip('[]').str.split()
    filtered_df['csi_data'] = filtered_df['csi_data'].apply(lambda x: [int(i) for i in x] if len(x) == 128 else None)
    filtered_df = filtered_df.dropna(subset=['csi_data'])
    
    # MultiIndex の列名を作成
    columns = [
        ('timestamp', ''), 
        *[(f'real', f'sc_{i}') for i in range(64)], 
        *[(f'imag', f'sc_{i}') for i in range(64)], 
        *[(f'amp', f'sc_{i}') for i in range(64)], 
        *[(f'phase', f'sc_{i}') for i in range(64)]
    ]
    
    # 各サブキャリアの値を計算して DataFrame に追加
    data = [
        [row['timestamp']] + 
        [row['csi_data'][2 * i] for i in range(64)] +  # real parts
        [row['csi_data'][2 * i + 1] for i in range(64)] +  # imag parts
        [np.sqrt(row['csi_data'][2 * i]**2 + row['csi_data'][2 * i + 1]**2) for i in range(64)] +  # amplitude
        [np.arctan2(row['csi_data'][2 * i + 1], row['csi_data'][2 * i]) for i in range(64)]  # phase
        for _, row in filtered_df.iterrows()
    ]
    
    # MultiIndex を持つ DataFrame を作成
    filtered_df = pd.DataFrame(data, columns=pd.MultiIndex.from_tuples(columns))

    return filtered_df

In [246]:
# 各locationごとに保存するための辞書を用意します
location_dfs = {}

# session_dfを順番に処理
for index, row in session_df.iterrows():
    session_id = row['session_id']
    location = row['location']
    start_time_jst_str = row['start_time_JST']
    end_time_jst_str = row['end_time_JST']

    start_time_jst = datetime.strptime(start_time_jst_str, '%Y-%m-%d %H:%M:%S')
    end_time_jst = datetime.strptime(end_time_jst_str, '%Y-%m-%d %H:%M:%S')
    print(f"Location: {location}, Start Time: {start_time_jst}, End Time: {end_time_jst}")

    session_dirpath = raw_data_dirpath / location / session_id 
    csi_data_filepath = session_dirpath / f'csi_{session_id}.csv'
    csi_data_df = pd.read_csv(csi_data_filepath)
    csi_data_df['timestamp'] = pd.to_datetime(pd.to_numeric(csi_data_df['timestamp'], errors='coerce'), unit='s')
    csi_data_df['sig_mode'] = pd.to_numeric(csi_data_df['sig_mode'], errors='coerce')

    filtered_df = filter_by_mac(csi_data_df)

    real = filtered_df['real'].values
    imag = filtered_df['imag'].values
    amp = filtered_df['amp'].values
    phase = filtered_df['phase'].values
    timestamp = filtered_df['timestamp'].apply(lambda x: x.timestamp()).tolist()

    target_fps = 100
    real_resampled = linear_interpolate_resampling(real, timestamp, start_time_jst.timestamp(), end_time_jst.timestamp(), target_fps)
    imag_resampled = linear_interpolate_resampling(imag, timestamp, start_time_jst.timestamp(), end_time_jst.timestamp(), target_fps)
    amp_resampled = linear_interpolate_resampling(amp, timestamp, start_time_jst.timestamp(), end_time_jst.timestamp(), target_fps)
    phase_resampled = linear_interpolate_resampling(phase, timestamp, start_time_jst.timestamp(), end_time_jst.timestamp(), target_fps)

    save_dirpath = root_data_dirpath / 'preprocessed' / 'csi' / 'resampled' / location / session_id
    save_dirpath.mkdir(parents=True, exist_ok=True)
    np.save(save_dirpath / 'real.npy', real_resampled)
    np.save(save_dirpath / 'imag.npy', imag_resampled)
    np.save(save_dirpath / 'amp.npy', amp_resampled)
    np.save(save_dirpath / 'phase.npy', phase_resampled)

Location: 512, Start Time: 2024-09-26 15:17:00, End Time: 2024-09-26 15:22:00
Location: 512, Start Time: 2024-09-26 15:24:00, End Time: 2024-09-26 15:29:00
Location: 512, Start Time: 2024-09-26 15:35:00, End Time: 2024-09-26 15:40:00
Location: 512, Start Time: 2024-09-26 15:47:00, End Time: 2024-09-26 15:52:00
Location: 511, Start Time: 2024-09-26 16:52:00, End Time: 2024-09-26 16:57:00
Location: 511, Start Time: 2024-09-26 17:01:00, End Time: 2024-09-26 17:06:00
Location: 511, Start Time: 2024-09-26 17:13:00, End Time: 2024-09-26 17:18:00
Location: 511, Start Time: 2024-09-26 17:23:00, End Time: 2024-09-26 17:28:00
Location: 514B, Start Time: 2024-09-26 17:58:00, End Time: 2024-09-26 18:03:00
Location: 514B, Start Time: 2024-09-26 18:05:00, End Time: 2024-09-26 18:10:00
Location: 514B, Start Time: 2024-09-26 18:12:00, End Time: 2024-09-26 18:17:00
Location: 514B, Start Time: 2024-09-26 18:19:00, End Time: 2024-09-26 18:24:00
